<a href="https://colab.research.google.com/github/amrKharroub/malaria-thick-smear-classifier/blob/main/TrainingModel0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle -q

In [2]:
import numpy as np
import os
import shutil
from zipfile import ZipFile
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import random
import json
import torch
import torch.nn as nn
from torch.cuda.amp import GradScaler
# import torchvision.models as models
from torchvision.models.resnet import resnet50
from torchvision.models.detection.faster_rcnn import FasterRCNN, FasterRCNN_ResNet50_FPN_V2_Weights, FastRCNNPredictor, FastRCNNConvFCHead
from torchvision.models.detection.backbone_utils import _resnet_fpn_extractor
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead

In [3]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  source = fn
  directory = os.path.join('/root', '.kaggle')
  os.mkdir(directory)
  destination = os.path.join(directory, 'kaggle.json')
  shutil.move(source, destination)

!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [4]:
!kaggle datasets download -d amrkharroub/aug-pv-data

with ZipFile("/content/aug-pv-data.zip", 'r') as zObject:
  zObject.extractall(path="/content/vivax_data")

100% 4.87G/4.88G [01:11<00:00, 92.0MB/s]
100% 4.88G/4.88G [01:11<00:00, 73.7MB/s]


In [5]:
base_dir = '/content/vivax_data/All_PvTk'
patients = os.listdir(base_dir)
pt_dirs = [os.path.join(base_dir, pt) for pt in patients]
images_dir = []
for pt in pt_dirs:
  images_dir.extend([os.path.join(pt, img) for img in os.listdir(pt)])
print(len(images_dir))
images_dir[0:10]

4519


['/content/vivax_data/All_PvTk/PvTk104/20190721_151349.jpg',
 '/content/vivax_data/All_PvTk/PvTk104/20190721_150613_aug.jpg',
 '/content/vivax_data/All_PvTk/PvTk104/20190721_151612.jpg',
 '/content/vivax_data/All_PvTk/PvTk104/20190721_150433.jpg',
 '/content/vivax_data/All_PvTk/PvTk104/20190721_152158.jpg',
 '/content/vivax_data/All_PvTk/PvTk104/20190721_151732_aug.jpg',
 '/content/vivax_data/All_PvTk/PvTk104/20190721_151349_aug.jpg',
 '/content/vivax_data/All_PvTk/PvTk104/20190721_150502.jpg',
 '/content/vivax_data/All_PvTk/PvTk104/20190721_151218_aug.jpg',
 '/content/vivax_data/All_PvTk/PvTk104/20190721_150052.jpg']

In [6]:
class CustomDataset(Dataset):
  def __init__(self, img_paths, labels_path, supervised=False, transform=True):
    self.img_paths = img_paths
    with open(labels_path, 'r')as f:
      self.labels = json.load(f)
    self.supervised = supervised
    self.transform = transform

  def apply_transformations(self, img):
    if self.transform:
      transformations = transforms.Compose([
          transforms.CenterCrop((3024, 3024)),
          transforms.ToTensor()
      ])
    else:
      transformations = transforms.ToTensor()
    tran_img = transformations(img)
    return tran_img
  
  def _get_target(self, img_path):
    img_name = img_path.split('k/')[1]
    boxes = torch.tensor(self.labels[img_name]['bbox'])
    target = {
        'boxes': boxes,
        'labels': torch.zeros(len(boxes), dtype=torch.int64)
    }
    return target
  
  def __len__(self):
    return len(self.img_paths)

  def __getitem__(self, idx):
    img_path = self.img_paths[idx]
    with Image.open(img_path) as image:
      image = self.apply_transformations(image)
    if self.supervised:
      target = self._get_target(img_path)
      return image, target
    else:
      return image


def collate_fn(batch):
    return tuple(zip(*batch))  

def split_data(img_paths, ratio):
    random.shuffle(img_paths)
    split_index = int(len(img_paths) * ratio)
    train_img_paths = img_paths[:split_index]
    validate_img_paths = img_paths[split_index:]
    return train_img_paths, validate_img_paths

In [7]:
train_images, valid_images = split_data(images_dir, 0.95)

train_dataset = CustomDataset(img_paths=train_images, labels_path='/content/vivax_data/annotationsV_v2.json', supervised=True, transform=False)
valid_dataset = CustomDataset(img_paths=valid_images, labels_path='/content/vivax_data/annotationsV_v2.json', supervised=False, transform=False)
loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

In [8]:
def custom_anchorgen():
    anchor_sizes = ((4,), (27,), (32,), (64,), (96,))
    aspect_ratios = ((0.75, 1.0, 1.5),) * len(anchor_sizes)
    return AnchorGenerator(anchor_sizes, aspect_ratios)

def _default_anchorgen():
    anchor_sizes = ((4,), (27,), (32,), (64,), (96,))
    aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
    return AnchorGenerator(anchor_sizes, aspect_ratios)

def fasterrcnn_resnet50_fpn_v2(
    *,
    weights=None,
    progress=True,
    num_classes=None,
    weights_backbone=None,
    trainable_backbone_layers=None,
    **kwargs
):

    if weights is not None:
        weights_backbone = None
        num_classes = len(weights.meta["categories"])
    
    trainable_backbone_layers = 3

    # weights_backbone = models.resnet50(pretrained=weights_backbone, progress=progress)
    backbone = resnet50(weights=weights_backbone, progress=progress) #is this pre-trained
    backbone = _resnet_fpn_extractor(backbone, trainable_backbone_layers, norm_layer=nn.BatchNorm2d)
    anchor_generator = custom_anchorgen()
    rpn_head = RPNHead(backbone.out_channels, anchor_generator.num_anchors_per_location()[0], conv_depth=2)
    box_head = FastRCNNConvFCHead(
        (backbone.out_channels, 7, 7), [256, 256, 256, 256], [1024], norm_layer=nn.BatchNorm2d
    )

    model = FasterRCNN(
        backbone,
        min_size=1024,
        max_size=1344,
        image_mean=[0.4512, 0.4089, 0.4721],
        image_std=[0.4051, 0.3715, 0.4202],
        num_classes=num_classes,
        rpn_anchor_generator=anchor_generator,
        rpn_head=rpn_head,
        box_head=box_head,
        **kwargs,
    )

    if weights is not None:
        model.load_state_dict(weights.get_state_dict(progress=progress))

    return model

In [9]:
#import torchvision
model = fasterrcnn_resnet50_fpn_v2(
        weights=FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
    )

# Get the number of input features 
in_features = model.roi_heads.box_predictor.cls_score.in_features
# define a new head for the detector with required number of classes
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2) 

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100%|██████████| 167M/167M [00:01<00:00, 140MB/s]


In [10]:
def train_one_epoch(model, optimizer, data_loader, device, scaler=None):
    model.train()
    
    losses = []

    
    for images, targets in data_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast(enabled=scaler is not None):
            loss_dict = model.forward(images, targets)
            loss = sum(loss for loss in loss_dict.values())
            
        losses.append(loss_dict)
        
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

    
    return losses


In [14]:
torch.cuda.empty_cache()

In [11]:
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

params = [p for p in model.parameters() if p.requires_grad]
# Define the optimizer.
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, nesterov=True)

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, 
            T_0=20,
            T_mult=1,
            verbose=True
        )


warmup_factor = 1.0 / 1000
warmup_iters = min(1000, len(loader) - 1)

lr_scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer, start_factor=warmup_factor, total_iters=warmup_iters
)

scaler = GradScaler()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

for epoch in range(2):

  losses = train_one_epoch(model, optimizer, loader, device, scaler)
  print(losses)
  if epoch == 0:
    lr_scheduler.step()
  else:
    scheduler.step()
  # Evaluate the model on the validation data
    # model.eval()
    # with torch.no_grad():
    #     for i, (images, labels) in enumerate(val_dataset):
    #         images = images.to(device)
    #         targets = targets.to(device)
            
    #         loss_dict = model(images, targets)

    # # Print the epoch number, the loss, and the accuracy
    # print(f"Epoch {epoch + 1}: loss={loss.item():.4f}, accuracy={accuracy:.4f}")

43,256,153 total parameters.
43,030,809 training parameters.
Epoch 00000: adjusting learning rate of group 0 to 1.0000e-03.
cuda
[{'loss_classifier': tensor(1.0067, device='cuda:0', grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0., device='cuda:0', grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.0727, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.1109, device='cuda:0', grad_fn=<DivBackward0>)}, {'loss_classifier': tensor(0.9983, device='cuda:0', grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0., device='cuda:0', grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.0788, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.1126, device='cuda:0', grad_fn=<DivBackward0>)}, {'loss_classifier': tensor(0.9783, device='cuda:0', grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0., device='cuda:0', grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.0783, device='cuda:0

In [12]:
!git clone https://github.com/amrKharroub/malaria-thick-smear-classifier.git

Cloning into 'malaria-thick-smear-classifier'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
Unpacking objects: 100% (7/7), 1.94 KiB | 995.00 KiB/s, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0


In [15]:
torch.save(model.state_dict(), '/content/malaria-thick-smear-classifier/model_state.pth')

# Save the entire model
torch.save(model, '/content/malaria-thick-smear-classifier/model.pth')

In [17]:
!ls

aug-pv-data.zip  malaria-thick-smear-classifier  sample_data  vivax_data


In [21]:
!git config --global user.email "amrkharroub@gmail.com"
!git config --global user.name "amrKharroub"

In [24]:
!git remote add stream https://github.com/amrKharroub/malaria-thick-smear-classifier.git

In [22]:
%cd /content/malaria-thick-smear-classifier
!git add .
!git commit -m 'trial models'


/content/malaria-thick-smear-classifier
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


In [26]:
!git config --global credential.helper cache

!git push stream main

fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
def evaluate(model, data_loader, device):
    model.eval()
    
    total_loss = 0.0
    num_images = 0
    
    with torch.no_grad():
        for images, targets in data_loader:
            images = images.to(device)
            targets = targets.to(device)
            
            loss_dict = model(images, targets)
            
            # Accumulate the total loss
            total_loss += sum(loss for loss in loss_dict.values()).item()
            num_images += len(images)
    
    # Calculate the average loss
    average_loss = total_loss / num_images
    
    return average_loss


In [ ]:

# Define the loss function and the optimizer
loss_fn = model.loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# Define the training loop
for epoch in range(10):

    # Train the model on the training data
    model.train()
    for i, (images, labels) in enumerate(train_dataset):

        # Forward pass
        predictions = model(images)

        # Calculate the loss
        loss = loss_fn(predictions, labels)

        # Backpropagate the loss
        optimizer.zero_grad()
        loss.backward()

        # Update the parameters
        optimizer.step()

    # Evaluate the model on the validation data
    model.eval()
    with torch.no_grad():
        for i, (images, labels) in enumerate(val_dataset):

            # Forward pass
            predictions = model(images)

            # Calculate the accuracy
            accuracy = (predictions["boxes"] == labels["boxes"]).sum().item() / len(labels["boxes"])

    # Print the epoch number, the loss, and the accuracy
    print(f"Epoch {epoch + 1}: loss={loss.item():.4f}, accuracy={accuracy:.4f}")
